# Vision Transformer


In [28]:
import h5py
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import h5py
from sklearn.metrics import roc_auc_score
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torch.nn import TransformerEncoder, TransformerEncoderLayer

In [39]:
    
# Load the photon and electron datasets
with h5py.File("photons.hdf5", "r") as f:
    photons_X = np.array(f["X"][:])
    photons_y = np.zeros(len(photons_X))

with h5py.File("electrons.hdf5", "r") as f:
    electrons_X = np.array(f["X"][:])
    electrons_y = np.ones(len(electrons_X))

MemoryError: Unable to allocate 1.90 GiB for an array with shape (249000, 32, 32, 2) and data type float32

In [38]:
# Combine the datasets and shuffle the samples
X = np.concatenate((photons_X, electrons_X))
y = np.concatenate((photons_y, electrons_y))




# Split into train and test sets
train_ratio = 0.8
train_size = int(train_ratio*len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

MemoryError: Unable to allocate 3.80 GiB for an array with shape (498000, 32, 32, 2) and data type float32

In [32]:
from tensorflow.keras.layers import Input, Dense, Dropout, LayerNormalization, MultiHeadAttention, GlobalAveragePooling1D
from tensorflow.keras.models import Model

def get_transformer_model(input_shape):
    inputs = Input(shape=input_shape)
    x = inputs
    for i in range(4):
        x = MultiHeadAttention(num_heads=8, key_dim=32)(x, x, x)
        x = LayerNormalization()(x)
        x = Dropout(0.1)(x)
        x = Conv1D(filters=64, kernel_size=3, padding='same', activation='relu')(x)
        x = LayerNormalization()(x)
        x = Dropout(0.1)(x)
    x = Reshape(target_shape=(input_shape[0]*64,))(x)
    x = Dense(units=128, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(units=1, activation='sigmoid')(x)
    model = Model(inputs=inputs, outputs=outputs)
    return model


In [44]:
from keras.layers import Add
from keras.layers import Input, Dense, Dropout, Reshape, Embedding, SpatialDropout1D, Conv1D, GlobalMaxPooling1D, Add, LayerNormalization
from keras.layers import MultiHeadAttention
from keras.models import Model
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping,  ModelCheckpoint



# Create and compile model
model = get_transformer_model(input_shape=(32, 2))
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])
# Train model
es = EarlyStopping(monitor='val_loss', mode='min', patience=10, verbose=1)
mc = ModelCheckpoint('best_model.h5', monitor='val_auc', mode='max', save_best_only=True, verbose=1)
history = model.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=32, callbacks=[es, mc])

Epoch 1/50


ValueError: in user code:

    File "C:\Users\USER\anaconda3\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\USER\anaconda3\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\USER\anaconda3\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\USER\anaconda3\lib\site-packages\keras\engine\training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\USER\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\USER\anaconda3\lib\site-packages\keras\engine\input_spec.py", line 295, in assert_input_compatibility
        raise ValueError(

    ValueError: Input 0 of laye